# GPS Signal Manipulation

In [1]:
# import statements go here.
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import os
import folium
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Parse Data

In [2]:
def read_file(path: str) -> pd.DataFrame:
    """
    Function predefined with the parameters needed to read the csv file
    for each person, receives the path to the file and returns a 
    a pandas dataframe
    """  
    names = ['lat', 'lon', 'date_time', 'duration']
    return pd.read_csv(path, sep=';', parse_dates=[2], infer_datetime_format=True, names=names, header=0)

In [3]:
file1 = 'Copy of Copy of person.1.csv'
file2 = 'Copy of Copy of person.2.csv'
file3 = 'Copy of Copy of person.3.csv'

person1 = read_file(file1)
person2 = read_file(file2)
person3 = read_file(file3)

Know lets take a peek into one of the datasets.

In [52]:
person1.shape

(549, 4)

In [53]:
person1.head()

,lat,lon,date_time,duration
0,-49.32696,-72.89073,2013-12-25 11:47:00-03:00,1186491
1,-49.32693,-72.89073,2013-12-25 12:13:00-03:00,4393711
2,-49.31661,-72.89890,2013-12-25 13:58:00-03:00,842939
3,-49.32716,-72.89072,2013-12-25 14:40:00-03:00,211887
4,-49.32688,-72.89085,2013-12-25 16:31:00-03:00,71166228


Use haversine to find the distance between 2 points in a sphere.

In [4]:
def haversine(point1: list, point2: list) -> float:
    """
    Function haversine receives a a pair of lists with (longitude, latitude)
    and returns a float which is the distance between two points on Earth in meters.
    
    It does so by assuming the Earth is a perfect sphere (which it isn't) 
    so it can lead to some errors, however this is trivial to compute.
    """
    lat1, lon1 = point1
    lat2, lon2 = point2
    
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2.) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.) ** 2    
    c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return 6.371e6*c

Find distance travelled by each person on every different row in the dataset

In [5]:
class calc_dist(object):
    """
    A helper class that stores the previous value seen or last position
    so that it is used with pandas dataframe method apply.
    """
    def __init__(self, past_pos):
        self.past_pos = past_pos
    
    def distance(self, curr_pos):
        result = haversine(curr_pos, self.past_pos)
        self.past_pos = curr_pos
        return result
        
    def reset(self, past_pos):
        self.past_pos = past_pos

In [6]:
dist = calc_dist(person1[['lat','lon']].loc[0].values)
person1['distance'] = person1.apply(lambda x: dist.distance([x.lat, x.lon]), axis=1)

dist.reset(person2[['lat','lon']].loc[0].values)
person2['distance'] = person2.apply(lambda x: dist.distance([x.lat, x.lon]), axis=1)

dist.reset(person3[['lat','lon']].loc[0].values)
person3['distance'] = person3.apply(lambda x: dist.distance([x.lat, x.lon]), axis=1)

> To have a better understanding of what is happening it will be helpful to see the date_time column transformed into other date time features.

In [7]:
def date_time_transform(df: pd.DataFrame, col='date_time') -> pd.DataFrame:
    """
    Function takes a dataframe and a string that specified the date time column
    to be transformed and returns a dataframe that includes 6 new columns
    dt_utc: date time normalized to utc
    month: month of dt_utc
    week: week of dt_utc
    weekday: weekday of dt_utc
    day: day of dt_utc
    hour: hour of dt_utc
    """
    df['dt_utc'] = pd.to_datetime(df[col], utc=True)
    df['year'] = df.dt_utc.dt.year
    df['month'] = df.dt_utc.dt.month
    df['week'] = df.dt_utc.dt.week
    df['weekday'] = df.dt_utc.dt.weekday
    df['day'] = df.dt_utc.dt.day
    df['hour'] = df.dt_utc.dt.hour
    df['utc'] = df[col].apply(lambda x: str(x).split(':')[2][-3:])
    return df

In [8]:
person1 = date_time_transform(person1)
person2 = date_time_transform(person2)
person3 = date_time_transform(person3)

Define a function to check basic stats about the datasets, particularly about time and distance.

In [9]:
def basic_stats(df: pd.DataFrame):
    """
    Function prints basic stats about the dataframe, like the number of location
    changes per day and the distance travelled.
    
    """
    time_stats = df[['year','month','day','weekday']].groupby(by=['year','month','day']).agg(['count']).reset_index().values
    time_stats_counts = pd.DataFrame(time_stats, columns=['year','month','day', 'day_count'])
    print('Time Stats')
    print(time_stats_counts.describe())
    print('\nDistance Stats')
    dist_stats = df[['year','month','day','distance']].groupby(by=['year','month','day']).agg(['mean','sum']).reset_index().values
    dist_stats_mean = pd.DataFrame(dist_stats, columns=['year','month','day', 'mean_dist','tot_dist'])
    print(dist_stats_mean.describe())

In [10]:
basic_stats(person1)

Time Stats
            year    month      day  day_count
count   79.00000 79.00000 79.00000   79.00000
mean  2013.91139  2.89873 14.58228    6.94937
std      0.28599  2.95953  8.39812    5.39681
min   2013.00000  1.00000  1.00000    1.00000
25%   2014.00000  1.00000  7.50000    4.00000
50%   2014.00000  2.00000 14.00000    5.00000
75%   2014.00000  3.00000 21.00000    8.00000
max   2014.00000 12.00000 31.00000   28.00000

Distance Stats
            year    month      day     mean_dist       tot_dist
count   79.00000 79.00000 79.00000      79.00000       79.00000
mean  2013.91139  2.89873 14.58228   29796.82388   240897.03924
std      0.28599  2.95953  8.39812  161208.46348  1310500.67835
min   2013.00000  1.00000  1.00000      54.29346      217.17383
25%   2014.00000  1.00000  7.50000    1545.72046    10253.08461
50%   2014.00000  2.00000 14.00000    2625.95563    13153.71247
75%   2014.00000  3.00000 21.00000    5672.44844    32578.40939
max   2014.00000 12.00000 31.00000 1424298.3372

### Data lookup

> Measurements can be noisy, and can change very slightly due to multiple factors, direct number to number comparisons between coordinates will not be helpful, so we need to take into account noise when we lookup if a person has visited or not previously a location.
> 
> For that we can specify an operating radius, that is, an area around the location we want to check. This area will be a circle with its center at the location and an specified radius.
>
> According to this [link](https://gis.stackexchange.com/questions/43617/what-is-the-maximum-theoretical-accuracy-of-gps) gps accurancy can be of around 7.8 mts with a 95% accuracy, so we predefine that number as our radius.

In [11]:
def user_at_location(point: list, df: pd.DataFrame, radius=7.8) -> int:
    """
    Function receives coordinates [lon, lat] and checks the pd.DataFrame to
    see if the user has visited such locations before. It does it calculating
    the distances between such point and all the coordinates in the dataframe
    and comparing it with radius, which is specified to considerate there
    might be noise in the measurements.
    
    returns the number of times such coordinates have been withing radius
    distance in the dataframe
    
    Note: This function could be a bottleneck as it needs to check the current location 
    to every other row in the dataframe.
    
    """
    distances = df.apply(lambda x: haversine([x.lat, x.lon], point), axis=1)
    return (distances <= radius).sum()

> To test the function we check a few coordinates within the persons journey and a few outside of it.

In [12]:
test1 = person1[['lat','lon']].values[0]
test2 = person2[['lat','lon']].values[1]
test3 = person3[['lat','lon']].values[2]

print('How many times has person 1 being in {}? {}'.format(test1, user_at_location(test1, person1)))
print('How many times has person 2 being in {}? {}'.format(test2, user_at_location(test2, person2)))
print('How many times has person 3 being in {}? {}'.format(test3, user_at_location(test3, person3)))

print('How many times has person 1 being in {}? {}'.format(test1, user_at_location(test1, person2)))
print('How many times has person 2 being in {}? {}'.format(test2, user_at_location(test2, person3)))
print('How many times has person 3 being in {}? {}'.format(test3, user_at_location(test3, person1)))

How many times has person 1 being in [-49.326958 -72.89073 ]? 4
How many times has person 2 being in [51.056984   3.7146811]? 5
How many times has person 3 being in [51.0542     4.4471655]? 1
How many times has person 1 being in [-49.326958 -72.89073 ]? 0
How many times has person 2 being in [51.056984   3.7146811]? 0
How many times has person 3 being in [51.0542     4.4471655]? 0


> This can also be assessed with a probability, that is, is we assume the error has a normal distribution with the previous 7.8 radius as 1.68 times our std deviation. With the previous data in mind, we would consider anything below 95% (or the threshold we wish to use) as not being the same location.

In [13]:
def prob_user_at_loc(point: list, df: pd.DataFrame, std=7.8) -> float:
    """
    This functions returns the Cumulative distribution function (CDF) will give you the probability that a random variable 
    is less than or equal to a certain real number. We assume the distance error is normally distributed
    and normalized using the standard deviation given from literature.
    """
    probs = df.apply(lambda x: st.norm.cdf(haversine([x.lat, x.lon], point) / std), axis=1)
    return probs

In [14]:
print('How many times has person 1 being in {}? {}'.format(test1, (prob_user_at_loc(test1, person1) <= 0.95).sum()))
print('How many times has person 2 being in {}? {}'.format(test2, (prob_user_at_loc(test2, person2) <= 0.95).sum()))
print('How many times has person 3 being in {}? {}'.format(test3, (prob_user_at_loc(test3, person3) <= 0.95).sum()))

print('How many times has person 1 being in {}? {}'.format(test1, (prob_user_at_loc(test1, person2) <= 0.95).sum()))
print('How many times has person 2 being in {}? {}'.format(test2, (prob_user_at_loc(test2, person3) <= 0.95).sum()))
print('How many times has person 3 being in {}? {}'.format(test3, (prob_user_at_loc(test3, person1) <= 0.95).sum()))

How many times has person 1 being in [-49.326958 -72.89073 ]? 5
How many times has person 2 being in [51.056984   3.7146811]? 6
How many times has person 3 being in [51.0542     4.4471655]? 1
How many times has person 1 being in [-49.326958 -72.89073 ]? 0
How many times has person 2 being in [51.056984   3.7146811]? 0
How many times has person 3 being in [51.0542     4.4471655]? 0


This gives use similar results to the previous aproach (though not equal).

### Common places detection 

The goal of this question, is to design an algorithm that allows us to distinguish the likely home locations of a user from his likely work locations.

Note that a person might have multiple home and work locations, or might not have a work location at all. Also note that the data might be noise, incorrect and/or incomplete.

Discuss your choice of algorithms, rules, methods, distance measures, etc.

> Lets see where this people have been, where they have travelled.

In [15]:
def display_map(df: pd.DataFrame, zoom=2, res=[800, 600], color='#0080bb', start=None):
    """
    Function that displays coordinates on a world map.
    """
    if start is None:
        start = df[['lat','lon']].values[0]
    
    map_disp = folium.Map(location=start, zoom_start=zoom, tiles='cartodbpositron', width=res[0], height=res[1])
    [folium.CircleMarker(x, radius=1, color=color, fill_color=color).add_to(map_disp) 
     for x in df[['lat','lon']].values]
    return map_disp

In [16]:
display_map(person1, zoom=1)

In [17]:
display_map(person2, zoom=9, color='#80bb00')

In [18]:
display_map(person3, zoom=9, color='#bb0080')

> To check their work / home for we might assume both work and home are places the user visits frequently if not daily, around the same hours, work more inclined to weekdays, home all days with perhaps more time on weekends.
>
> But first, we need to reduce the spatial space, that is, try to cluster the gps measurements that we believe might be the from the same place.
> 
> I'll use DBSCAN as it is a clustering method that creates clusters by density, for which an eps or distance can be specified, it also has the haversine metric that I have been using before.
>
> I have some ideas for including the time spent in one location as weight for that location (more time more weight).

In [19]:
def cluster_locations(df: pd.DataFrame, eps: float, min_samples: int, weight=None):
    """
    Function to return a list that identifies clusters based on geographic density, 
    that is, points that are close enough to one another to be considered in a same cluster.
    
    Labels equal to "0" are coordinates that are not part of a cluster.
    """
    mts_per_radian = 6371008.8
    eps_rad = eps / mts_per_radian
    coord_rad = np.radians(df[['lat','lon']].values)
    db = DBSCAN(eps=eps_rad, min_samples=min_samples, algorithm='ball_tree', metric='haversine', n_jobs=-1)
    
    if weight is not None:
        clusters = db.fit_predict(coord_rad, sample_weight=weight)
    else:
        clusters = db.fit_predict(coord_rad)
    
    return clusters+1


def cluster_coords(df, clusters):
    """
    A helper function that will group all coordinates of the same cluster in the 
    same list, creating a list of lists of clusters
    """
    coords = df[['lat','lon']].values
    num_clusters = set(clusters)
    df_clusters = pd.Series([coords[clusters == n] for n in num_clusters if n != 0])
    return df_clusters


def get_centermost_point(cluster):
    """
    Helper function that returns the center most coordinate from a group of coordinates.
    """
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)


def get_representative_points(df, df_clusters):
    """
    Function that gets the rows in the original dataframe that correspond to the most
    representative point for each cluster.
    """
    center_points = df_clusters.map(get_centermost_point)
    lats, lons = zip(*center_points)
    rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
    
    rs = rep_points.apply(lambda row: df[np.logical_and(df['lat']==row['lat'], df['lon']==row['lon'])].iloc[0], axis=1)
    
    return rs 


def display_map_clust(df: pd.DataFrame, rs, zoom=2, res=[800, 600], color='#000000', start=None):
    """
    Function that plots all coordinates along with the clusters as an area around the most
    representative coordinates for each cluster.
    """
    if start is None:
        start = df[['lat','lon']].values[0]
    
    map_disp = folium.Map(location=start, zoom_start=zoom, tiles='cartodbpositron', width=res[0], height=res[1])
    [folium.CircleMarker(x, radius=1, color=color, fill_color=color).add_to(map_disp) 
     for x in df[['lat','lon']].values]
    
    [folium.CircleMarker(y, radius=30, color='#777777', fill_color='#777777', line_opacity=0.1, fill_opacity=0.1).add_to(map_disp) 
     for y in rs[['lat','lon']].values]
    
    return map_disp

In [20]:
clusters1 = cluster_locations(person1, eps=7.8, min_samples=2)
df_clusters1 = cluster_coords(person1, clusters1)
rs1 = get_representative_points(person1, df_clusters1)

display_map_clust(person1, rs1, zoom=3)

In [21]:
clusters2 = cluster_locations(person2, eps=7.8, min_samples=2)
df_clusters2 = cluster_coords(person2, clusters2)
rs2 = get_representative_points(person2, df_clusters2)

display_map_clust(person2, rs2, zoom=9)

In [22]:
clusters3 = cluster_locations(person3, eps=7.8, min_samples=2)
df_clusters3 = cluster_coords(person3, clusters3)
rs3 = get_representative_points(person3, df_clusters3)

display_map_clust(person3, rs3, zoom=9)

> Proceed to see which clusters are the most popular for each person and see if we can infer some information from them.

In [23]:
person1['cluster'] = cluster_locations(person1, eps=7.8, min_samples=2)
person2['cluster'] = cluster_locations(person2, eps=7.8, min_samples=2)
person3['cluster'] = cluster_locations(person3, eps=7.8, min_samples=2)

In [24]:
filter1 = person1[person1.cluster!=0].groupby(by=['cluster']).count().reset_index().sort_values('lat', ascending=False).head(3).cluster
person1[person1.cluster.isin(filter1)].groupby(['cluster','weekday']).count()

lat  lon  date_time  duration  distance  dt_utc  year  month  \
cluster weekday                                                                 
11      0         18   18         18        18        18      18    18     18   
        1         12   12         12        12        12      12    12     12   
        2         11   11         11        11        11      11    11     11   
        3         14   14         14        14        14      14    14     14   
        4         12   12         12        12        12      12    12     12   
        5         18   18         18        18        18      18    18     18   
        6         14   14         14        14        14      14    14     14   
16      0          6    6          6         6         6       6     6      6   
        1          7    7          7         7         7       7     7      7   
        2          7    7          7         7         7       7     7      7   
        3          4    4          4         4         4       4     4      4   
        4          6    6          6         6         6       6     6      6   
27      0          6    6          6         6         6       6     6      6   
        1          5    5          5         5         5       5     5      5   
        2          6    6          6         6         6       6     6      6   
        3          5    5          5         5         5       5     5      5   
        4         10   10         10        10        10      10    10     10   

                 week  day  hour  utc  
cluster weekday                        
11      0          18   18    18   18  
        1          12   12    12   12  
        2          11   11    11   11  
        3          14   14    14   14  
        4          12   12    12   12  
        5          18   18    18   18  
        6          14   14    14   14  
16      0           6    6     6    6  
        1           7    7     7    7  
        2           7    7     7    7  
        3           4    4     4    4  
        4           6    6     6    6  
27      0           6    6     6    6  
        1           5    5     5    5  
        2           6    6     6    6  
        3           5    5     5    5  
        4          10   10    10   10

> From this 3 clusters we could think cluster #11 is the home from the person as there are visits every day to it (Monday to Sunday) the other 2 could be workplaces, or work and a place to have lunch as they are only visited on from Monday to Friday

In [25]:
filter1 = person1.cluster.isin([11,27,16])
clusters1 = cluster_locations(person1[filter1], eps=7.8, min_samples=2)
df_clusters1 = cluster_coords(person1[filter1], clusters1)
rs1 = get_representative_points(person1[filter1], df_clusters1)

display_map_clust(person1[filter1], rs1, zoom=12)

In [26]:
filter2 = person2[person2.cluster!=0].groupby(by=['cluster']).count().reset_index().sort_values('lat', ascending=False).head(3).cluster
person2[person2.cluster.isin(filter2)].groupby(['cluster','weekday']).count()

lat  lon  date_time  duration  distance  dt_utc  year  month  \
cluster weekday                                                                 
3       0         23   23         23        23        23      23    23     23   
        1         15   15         15        15        15      15    15     15   
        2         16   16         16        16        16      16    16     16   
        3         24   24         24        24        24      24    24     24   
        4         19   19         19        19        19      19    19     19   
        5         25   25         25        25        25      25    25     25   
        6         25   25         25        25        25      25    25     25   
5       0         10   10         10        10        10      10    10     10   
        1          8    8          8         8         8       8     8      8   
        2          8    8          8         8         8       8     8      8   
        3         10   10         10        10        10      10    10     10   
        4          9    9          9         9         9       9     9      9   
        5          3    3          3         3         3       3     3      3   
39      0          7    7          7         7         7       7     7      7   
        1          7    7          7         7         7       7     7      7   
        2          4    4          4         4         4       4     4      4   
        3         12   12         12        12        12      12    12     12   
        4          7    7          7         7         7       7     7      7   
        5          2    2          2         2         2       2     2      2   

                 week  day  hour  utc  
cluster weekday                        
3       0          23   23    23   23  
        1          15   15    15   15  
        2          16   16    16   16  
        3          24   24    24   24  
        4          19   19    19   19  
        5          25   25    25   25  
        6          25   25    25   25  
5       0          10   10    10   10  
        1           8    8     8    8  
        2           8    8     8    8  
        3          10   10    10   10  
        4           9    9     9    9  
        5           3    3     3    3  
39      0           7    7     7    7  
        1           7    7     7    7  
        2           4    4     4    4  
        3          12   12    12   12  
        4           7    7     7    7  
        5           2    2     2    2

> Something similar could be inferred here is the home from the person as there are visits every day to it (Monday to Sunday) the other 2 could be workplaces, or work and a place to have lunch as they are only visited on from Monday to Saturday

In [27]:
filter2 = person2.cluster.isin([3,5,39])
clusters2 = cluster_locations(person2[filter2], eps=7.8, min_samples=2)
df_clusters2 = cluster_coords(person2[filter2], clusters2)
rs2 = get_representative_points(person2[filter2], df_clusters2)

display_map_clust(person2[filter2], rs2, zoom=12)

In [28]:
filter3 = person3[person3.cluster!=0].groupby(by=['cluster']).count().reset_index().sort_values('lat', ascending=False).head(3).cluster
person3[person3.cluster.isin(filter3)].groupby(['cluster','weekday']).count()

lat  lon  date_time  duration  distance  dt_utc  year  month  \
cluster weekday                                                                 
3       0         13   13         13        13        13      13    13     13   
        1         14   14         14        14        14      14    14     14   
        2         11   11         11        11        11      11    11     11   
        3         11   11         11        11        11      11    11     11   
        4         10   10         10        10        10      10    10     10   
        5         19   19         19        19        19      19    19     19   
        6          9    9          9         9         9       9     9      9   
7       0          3    3          3         3         3       3     3      3   
        1          3    3          3         3         3       3     3      3   
        2          1    1          1         1         1       1     1      1   
        3          3    3          3         3         3       3     3      3   
        4          3    3          3         3         3       3     3      3   
        5          4    4          4         4         4       4     4      4   
        6          5    5          5         5         5       5     5      5   
12      0          6    6          6         6         6       6     6      6   
        1          5    5          5         5         5       5     5      5   
        2          6    6          6         6         6       6     6      6   
        3          5    5          5         5         5       5     5      5   
        4          6    6          6         6         6       6     6      6   

                 week  day  hour  utc  
cluster weekday                        
3       0          13   13    13   13  
        1          14   14    14   14  
        2          11   11    11   11  
        3          11   11    11   11  
        4          10   10    10   10  
        5          19   19    19   19  
        6           9    9     9    9  
7       0           3    3     3    3  
        1           3    3     3    3  
        2           1    1     1    1  
        3           3    3     3    3  
        4           3    3     3    3  
        5           4    4     4    4  
        6           5    5     5    5  
12      0           6    6     6    6  
        1           5    5     5    5  
        2           6    6     6    6  
        3           5    5     5    5  
        4           6    6     6    6

> Similarly here.

In [29]:
filter3 = person3.cluster.isin([3,7,12])
clusters3 = cluster_locations(person3[filter3], eps=7.8, min_samples=2)
df_clusters3 = cluster_coords(person3[filter3], clusters3)
rs3 = get_representative_points(person3[filter3], df_clusters3)

display_map_clust(person3[filter3], rs3, zoom=9)